#### Load Libraries and Demo Codes

In [1]:
if True:
    from julia.api import Julia
    jl = Julia(compiled_modules=False)

import julia; julia.install(quiet=True)
from julia import Main

import sympy as sp
import numpy as np
from numpy.linalg import qr, svd
from sklearn.decomposition import PCA

import time

import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

import param
import holoviews as hv
import panel as pn

pn.extension('katex', 'plotly')
hv.extension('plotly', 'bokeh', logo=False)

%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [2]:
%%html
<script src="https://cdnjs.cloudflare.com/ajax/libs/paper.js/0.12.18/paper-full.min.js"></script>

In [3]:
%%julia
using Pkg; Pkg.activate("../GenLinAlgProblems")
using GenLinAlgProblems, LinearAlgebra, RowEchelon, Printf, Latexify, LaTeXStrings, Random, SymPy

  Activating project at `~/NOTEBOOKS/elementary-linear-algebra/GenLinAlgProblems`


In [4]:
#%output backend="plotly"
def spherical_to_cartesian(r, theta_deg, phi_deg):
    theta = np.radians(theta_deg)
    phi = np.radians(phi_deg)
    return r * np.array([
        np.sin(theta) * np.cos(phi),
        np.sin(theta) * np.sin(phi),
        np.cos(theta)
    ])
def rotation_matrix(axis, theta):
    """
    Returns the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / np.linalg.norm(axis)
    a = np.cos(theta / 2.0)
    b, c, d = -axis * np.sin(theta / 2.0)
    return np.array([
        [a*a + b*b - c*c - d*d, 2*(b*c - a*d),     2*(b*d + a*c)],
        [2*(b*c + a*d),     a*a + c*c - b*b - d*d, 2*(c*d - a*b)],
        [2*(b*d - a*c),     2*(c*d + a*b),     a*a + d*d - b*b - c*c]
    ])
def project_to_plane(v, normal):
    n = normal / np.linalg.norm(normal)
    return v - np.dot(v, n) * n

In [5]:
#%output backend="plotly"
def dynamic_name(group, name):
    """
    Generates a standardized dynamic trace name.

    Args:
        group (str): Type of the trace ("plane", "vector", "arc", etc.)
        name (str): Descriptive name of the trace.

    Returns:
        str: Combined dynamic trace name.
    """
    return f"dynamic:{group}:{name}"
def replace_traces(fig, new_traces, group="dynamic"):
    """
    Safely replaces all traces in the figure that belong to the given dynamic group.
    """
    prefix   = group if group.endswith(":") else f"{group}:"
    fig.data = [trace for trace in fig.data if not (hasattr(trace, "name") and trace.name.startswith("dynamic:"))]
    fig.add_traces(new_traces)
def create_plane(origin, v1, v2, size=2.0):
    v1 = v1 / np.linalg.norm(v1)
    v2 = v2 / np.linalg.norm(v2)
    points = [origin + a * v1 + b * v2 for a in [-size, size] for b in [-size, size]]
    x = np.array([p[0] for p in points]).reshape(2, 2)
    y = np.array([p[1] for p in points]).reshape(2, 2)
    z = np.array([p[2] for p in points]).reshape(2, 2)
    return x, y, z
def plot_vector_pair(u_vec, v_vec, scale=1.6, u_color='darkred', v_color='blue', name_prefix='principal'):
    return [
        go.Scatter3d(x=[0, u_vec[0]*scale], y=[0, u_vec[1]*scale], z=[0, u_vec[2]*scale],
                     mode='lines', line=dict(color=u_color, width=6),
                     name=dynamic_name("vector", f"{name_prefix} u"),
                     showlegend=False),
        go.Scatter3d(x=[0, v_vec[0]*scale], y=[0, v_vec[1]*scale], z=[0, v_vec[2]*scale],
                     mode='lines', line=dict(color=v_color, width=6),
                     name=dynamic_name("vector", f"{name_prefix} v"),
                     showlegend=False)
    ]
def angle_arc(u, v, origin=None, color='black', name_prefix="arc"):
    if origin is None:
        origin = np.zeros(3)

    u_norm = u / np.linalg.norm(u)
    v_norm = v / np.linalg.norm(v)

    angle = np.arccos(np.clip(np.dot(u_norm, v_norm), -1, 1))

    if np.isclose(angle, 0) or np.isclose(angle, np.pi):
        return None  # No arc needed

    # Build orthonormal basis u_norm, w in plane
    w = v_norm - np.dot(v_norm, u_norm) * u_norm
    if np.linalg.norm(w) < 1e-6:
        return None
    w /= np.linalg.norm(w)

    n_points = 30
    angles = np.linspace(0, angle, n_points)
    arc_points = []

    for θ in angles:
        p = np.cos(θ) * u_norm + np.sin(θ) * w
        arc_points.append(p + origin)

    arc_points = np.array(arc_points)

    return go.Scatter3d(
        x=arc_points[:, 0],
        y=arc_points[:, 1],
        z=arc_points[:, 2],
        mode='lines',
        line=dict(color=color, width=4, dash='dash'),
        showlegend=False,
        name=dynamic_name("arc", name_prefix)
    )
def plot_intersection_line(u1, u2, v1, v2, scale=2.5, name_prefix="Intersection Line"):
    nU = np.cross(u1, u2)
    nV = np.cross(v1, v2)
    line_dir = np.cross(nU, nV)

    if np.linalg.norm(line_dir) < 1e-6:
        return None  # No well-defined intersection

    line_dir /= np.linalg.norm(line_dir)

    return go.Scatter3d(
        x=[-scale*line_dir[0], scale*line_dir[0]],
        y=[-scale*line_dir[1], scale*line_dir[1]],
        z=[-scale*line_dir[2], scale*line_dir[2]],
        mode='lines',
        line=dict(color='black', width=3, dash='dash'),
        name=dynamic_name("intersection", name_prefix),
        showlegend=False  # Hide from legend to avoid jitter
    )

In [6]:
class PrincipalAnglePlayer(pn.viewable.Viewer):
    offset  = param.Number(default=0.0,  bounds=(-5.0, 5.0), label="Z Offset for Plane V")
    theta_1 = param.Integer(default=0,  bounds=(-1800, 1800), label="theta_1")  # 10*degrees
    theta_2 = param.Integer(default=90, bounds=(-1800, 1800), label="theta_2")  # 10*degrees

    r     = param.Number(default=3.5, bounds=(1.0, 10.0), label="Camera Distance")
    theta = param.Number(default=45,  bounds=(0, 360),    label="Azimuth (θ°)")
    phi   = param.Number(default=30,  bounds=(-90, 90),   label="Elevation (φ°)")

    def __init__(self, **params):
        super().__init__(**params)
        self.fig_pane = pn.pane.Plotly(height=650, config={'responsive': True})
        self.distances_pane = pn.pane.Markdown("", sizing_mode="stretch_width")

        self._locked_camera = None
        self.lock_button = pn.widgets.Button(name="Lock Current Camera", button_type="primary", width=200)
        self.lock_button.on_click(self._lock_camera_from_plot)

        self.param.watch(self._update_data_only, ['theta_1', 'theta_2', 'offset'])
        self.param.watch(self._update_camera_only, ['r', 'theta', 'phi'])

        self._current_camera = self._camera_eye()
        self._update_data_only()

    def _svd_from_planes(self, u1, u2, v1, v2):
        U, _ = np.linalg.qr(np.stack([u1, u2], axis=1))
        V, _ = np.linalg.qr(np.stack([v1, v2], axis=1))
        return np.linalg.svd(U.T @ V), U, V

    def compute_principal_vectors_and_angles(self, u1, u2, v1, v2):
        (W1, s, W2T), U, V = self._svd_from_planes(u1, u2, v1, v2)
        principal_U = U @ W1
        principal_V = V @ W2T.T
        angles = np.arccos(np.clip(s, -1, 1))
        return principal_U, principal_V, np.degrees(angles)

    def _camera_eye(self):
        θ, φ = np.radians(0.1*self.theta), np.radians(0.1*self.phi)
        zoom_factor = self.r
        x = zoom_factor * np.cos(θ) * np.cos(φ)
        y = zoom_factor * np.sin(θ) * np.cos(φ)
        z = zoom_factor * np.sin(φ)
        return dict(x=x, y=y, z=z)

    def _lock_camera_from_plot(self, _=None):
        if self.fig_pane.object and hasattr(self.fig_pane.object.layout.scene, "camera"):
            cam = self.fig_pane.object.layout.scene.camera.eye
            self._locked_camera = dict(x=cam.x, y=cam.y, z=cam.z)

    def _update_camera_only(self, *_):
        if self._locked_camera:
            self._current_camera = self._locked_camera
        else:
            self._current_camera = self._camera_eye()

        if self.fig_pane.object:
            self.fig_pane.object.update_layout(scene_camera=dict(eye=self._current_camera))

    def _update_data_only(self, *_):
        θ1 = np.radians(self.theta_1 * 0.1)
        θ2 = np.radians(self.theta_2 * 0.1)

        u1 = np.array([1, 0, 0])
        u2 = np.array([0, 1, 0])
        origin = np.zeros(3)

        v1 = np.array([np.cos(θ1), np.sin(θ1), 0])
        v1_perp = np.array([-np.sin(θ1), np.cos(θ1), 0])
        v2 = np.cos(θ2) * v1_perp + np.sin(θ2) * np.array([0, 0, 1])

        xU, yU, zU = create_plane(origin, u1, u2)
        xV, yV, zV = create_plane(origin + np.array([0, 0, self.offset]), v1, v2)

        principal_U, principal_V, angles = self.compute_principal_vectors_and_angles(u1, u2, v1, v2)

        # --- Create all new traces ---
        traces = [
            go.Surface(x=xU, y=yU, z=zU, opacity=0.85, showscale=False,
                       colorscale=[[0, '#989F7A'], [1, '#989F7A']],
                       name=dynamic_name("plane", "Plane U"), showlegend=False),
            go.Surface(x=xV, y=yV, z=zV, opacity=0.65, showscale=False,
                       colorscale=[[0, '#bbbbbb'], [1, '#bbbbbb']],
                       name=dynamic_name("plane", "Plane V"), showlegend=False)
        ]

        traces += plot_vector_pair(principal_U[:, 0], principal_V[:, 0], name_prefix='p1')
        traces += plot_vector_pair(principal_U[:, 1], principal_V[:, 1], name_prefix='p2')

        arc1 = angle_arc(principal_U[:, 0], principal_V[:, 0], origin=origin, color='black', name_prefix="θ₁ arc")
        arc2 = angle_arc(principal_U[:, 1], principal_V[:, 1], origin=origin, color='black', name_prefix="θ₂ arc")
        if arc1: traces.append(arc1)
        if arc2: traces.append(arc2)

        inter_line = plot_intersection_line(u1, u2, v1, v2)
        if inter_line: traces.append(inter_line)

        title_text = f"Principal Angles: θ₁ = {angles[0]:.2f}°, θ₂ = {angles[1]:.2f}°"
        # --- Full new figure (flicker-free), reusing locked or current camera ---
        camera_settings = dict(
            eye=self._locked_camera if self._locked_camera else self._camera_eye(),
            projection=dict(type="orthographic")
        )

        fig = go.Figure(data=traces)
        fig.update_layout(
            title=title_text,
            scene=dict(
                aspectmode='cube',
                xaxis=dict(tickmode='array', tickvals=[-2, -1, 0, 1, 2], range=[-2.5, 2.5], title='X'),
                yaxis=dict(tickmode='array', tickvals=[-2, -1, 0, 1, 2], range=[-2.5, 2.5], title='Y'),
                zaxis=dict(tickmode='array', tickvals=[-1, 0, 1, 2, 3], range=[-1.5, 3.0], title='Z'),
                camera=camera_settings
            ),
            legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5, font=dict(size=12)),
            margin=dict(l=10, r=10, t=40, b=10),
            showlegend=True
        )

        self.fig_pane.object = fig
        self.distances_pane.object = self._compute_distances_markdown()

    def _compute_distances_markdown(self):
        θ1 = np.radians(self.theta_1 * 0.1)
        θ2 = np.radians(self.theta_2 * 0.1)

        u1 = np.array([1, 0, 0])
        u2 = np.array([0, 1, 0])

        v1 = np.array([np.cos(θ1), np.sin(θ1), 0])
        v1_perp = np.array([-np.sin(θ1), np.cos(θ1), 0])
        v2 = np.cos(θ2) * v1_perp + np.sin(θ2) * np.array([0, 0, 1])

        (_, s, _), _, _ = self._svd_from_planes(u1, u2, v1, v2)
        s = np.clip(s, 0, 1)
        angles = np.arccos(s)

        spectral   = np.sin(np.max(angles))
        chordal    = np.sqrt(np.sum(np.sin(angles)**2))
        projection = np.sqrt(2) * chordal
        geodesic   = np.sqrt(np.sum(angles**2))

        text = f"""
        **Distances:**
        - **Spectral Distance:**   {spectral:20.4f}
        - **Chordal Distance:**    {chordal:20.4f}
        - **Projection Distance:** {projection:20.4f}
        - **Geodesic Distance:**   {geodesic:20.4f} radians
        """
        return text

    def __panel__(self):
        return pn.Row(
            pn.Column(
                pn.pane.Markdown("## Principal Angle Visualizer"),
                self.fig_pane,
                width=500,
            ),
            pn.Column(
                r"## Controls: θ₁ (Intersection), θ₂ (Inclination).",
                self.distances_pane,
                pn.Spacer(height=20),
                self.lock_button,
                pn.Spacer(height=20),

                "theta_1", pn.widgets.Player.from_param(self.param.theta_1, width=250, interval=10, step=5, start=-1800, end=1800, loop_policy='reflect'),
                "theta_2", pn.widgets.Player.from_param(self.param.theta_2, width=250, interval=10, step=5, start=-1800, end=1800, loop_policy='reflect'),
                width=500,
            ),
        )

In [7]:
from scipy.linalg import subspace_angles

def angle_to_vector(angle_deg, base_vector, orthogonal_direction):
    theta = np.radians(angle_deg)
    base = base_vector / np.linalg.norm(base_vector)
    ortho = orthogonal_direction / np.linalg.norm(orthogonal_direction)
    return np.cos(theta) * base + np.sin(theta) * ortho
def generate_subspaces(theta1_deg, theta2_deg):
    """
    Constructs a fixed 3D subspace V, and a 2D subspace U that varies with angles θ₁ and θ₂.
    """
    # Fixed 3D subspace V = span(e₁, e₂, e₅)
    v1 = np.array([1, 0, 0, 0, 0])  # e1
    v2 = np.array([0, 1, 0, 0, 0])  # e2
    v3 = np.array([0, 0, 0, 0, 1])  # e5
    V = np.stack([v1, v2, v3], axis=1)

    # Rotating 2D subspace U = span(u₁, u₂)
    e1 = np.array([1, 0, 0, 0, 0])
    e1_perp = np.array([0, 0, 1, 0, 0])  # rotates in (e₁, e₃)
    e2 = np.array([0, 1, 0, 0, 0])
    e2_perp = np.array([0, 0, 0, 1, 0])  # rotates in (e₂, e₄)

    u1 = angle_to_vector(theta1_deg, e1, e1_perp)
    u2 = angle_to_vector(theta2_deg, e2, e2_perp)
    U = np.stack([u1, u2], axis=1)

    return U, V
def projection_distance(U, V):
    """
    Standard projection distance: spectral norm of P_U - P_V
    May be dominated by static orthogonal components (e.g., v3 = e5).
    """
    PU = U @ U.T
    PV = V @ V.T
    return np.linalg.norm(PU - PV, ord=2)
def compute_distance_metrics(U, V):
    theta     = subspace_angles(U, V)
    sin_theta = np.sin(theta)

    return {
        'chordal':         np.linalg.norm(sin_theta),
        'spectral':        np.max(sin_theta),
        'projection':      projection_distance(U, V),
        'frobenius':       np.sqrt(2 * np.sum(sin_theta**2)),
        'geodesic':        np.linalg.norm(theta)
    }

In [8]:
def ex_1():
    # Construct a nearly rank-1 matrix A
    np.random.seed(42)
    n = 10
    a1 = np.random.randn(n)
    a2 = a1 + 1e-14 * np.random.randn(n)  # almost identical
    A = np.column_stack([a1, a2])

    # QR decomposition
    Q, R = np.linalg.qr(A)
    q1, q2 = Q[:, 0], Q[:, 1]

    # SVD decomposition
    U, S, _ = np.linalg.svd(A, full_matrices=False)
    u1 = U[:, 0]

    # Diagnostics
    angle_1 = np.degrees(np.arccos(np.clip(np.dot(q1, u1), -1.0, 1.0)))
    angle_2 = np.degrees(np.arccos(np.clip(np.dot(q2, u1), -1.0, 1.0)))
    orth_qr = np.linalg.norm(Q.T @ Q - np.eye(2), ord=np.inf)
    print("Given vectors a1 and a2 are nearly identical:\n")
    print( "\nQR returns two orthogonal vectors")
    print(f"    ‖QᵀQ − I‖∞      = {orth_qr:.2e}")
    print(f"    diagonal elements of R: {R[0,0]:.6f}, {R[1,1]:.2e}")
    print( "\nSVD singular values reveal the near rank deficiency")
    print(f"    σ₁ = {S[0]:.2e},   σ₂ = {S[1]:.2e}")
    print("\nCompare the angles of the QR vectors q₁ and q₂ with the singular vector u₁")
    print(f"    angle(q₁, u₁) = {angle_1:.6f}°")
    print(f"    angle(q₂, u₁) = {angle_2:.6f}°")


In [9]:
def qr_vs_svd_stacked_annotated(min_exp=-5, max_deg=90.0, step_exp=0.05):
    input_angles = []
    q1_u1_angles = []
    sigma_ratios = []
    r22_vals     = []

    log_degrees = np.arange(min_exp, np.log10(max_deg) + step_exp, step_exp)
    for log_deg in log_degrees:
        theta_deg = max(10 ** log_deg, 1e-15)
        theta_rad = np.radians(theta_deg)

        a1 = np.array([1.0, 0.0])
        a2 = np.array([np.cos(theta_rad), np.sin(theta_rad)])
        A  = np.column_stack((a1, a2))

        Q, R = qr(A)
        U_svd, S, _ = svd(A)

        r22_vals.append(abs(R[1, 1]))

        # angle between q₁ and u₁
        cosθ = np.clip(np.dot(Q[:, 0], U_svd[:, 0]), -1.0, 1.0)
        angle_q1_u1 = np.degrees(np.arccos(cosθ))

        sigma_ratio = S[1] / S[0]

        input_angles.append(theta_deg)
        q1_u1_angles.append(angle_q1_u1)
        sigma_ratios.append(sigma_ratio)

    # ====================================================== Graphs
    o = hv.opts.Curve(height=120, width=500, show_grid=True,
                      #min_border=0, min_border_top=0, min_border_bottom=0,
                      logx=True, xlim=(1e-5, max_deg), yticks=4,
                      tools=['hover'])

    angle_curve = hv.Curve((input_angles, q1_u1_angles),
                           'θ(a₁, a₂) [deg]', 'θ(q₁, u₁) [deg]').opts(color='crimson', xaxis=None)

    r22_curve = hv.Curve((input_angles, r22_vals),
                         'θ(a₁, a₂) [deg]', 'R₂₂').opts( color='blue', ylabel='R₂₂', xaxis=None)

    sigma_curve = hv.Curve((input_angles, sigma_ratios),# label='computed curve')\
                           'θ(a₁, a₂) [deg]', 'σ₂ / σ₁',)\
                           .opts(color='navy')

    # Compute theoretical curve for comparison
    theta_rads         = np.radians(input_angles)
    theoretical_ratios = np.tan(theta_rads / 2)

    theoretical_curve = hv.Curve((input_angles, theoretical_ratios),
                                 'θ(a₁, a₂) [deg]', 'σ₂ / σ₁',# label='theoretical tan(θ/2)'
                                ).opts(color='green')

    # --- Annotations ---
    elbow_angle = 1.0  # where things start to diverge
    vline       = hv.VLine(elbow_angle).opts(color='gray', line_dash='dashed', line_width=2)

    label = hv.Text(elbow_angle * 1.05, 30.0, "↖ elbow region").opts(
        text_align='left', text_font_size='10pt', text_color='gray'
    )

    return pn.Column(
        (angle_curve * vline * label).opts(o).opts(height=100),
        (r22_curve * vline).opts(o).opts(height=100),
        (sigma_curve * vline ).opts(o) #.opts(legend_position='bottom')
    )

In [10]:
class CombinedDistanceViewer(pn.viewable.Viewer):
    def __init__(self, resolution=91, **params):
        super().__init__(**params)

        self.θ_vals          = np.linspace(0, 90, resolution)
        self.metrics         = ['chordal', 'spectral', 'projection', 'geodesic', 'frobenius']
        self.metric_selector = pn.widgets.Select(name="Metric (Heatmap)", options=self.metrics, value='chordal')
        self.fix_toggle      = pn.widgets.RadioButtonGroup(name='Fix Angle', options=['θ₁ fixed', 'θ₂ fixed']) #, button_type='primary')
        self.angle_slider    = pn.widgets.FloatSlider(name="Fixed Angle (°)", start=0, end=90, step=1, value=30)

        self.data = self._precompute_data()

        self.plot_panel = pn.bind(self._make_plot, self.metric_selector, self.fix_toggle, self.angle_slider)

        self.layout = pn.Column(
            "## Combined Heatmap + Curve Viewer",
            pn.Row(self.metric_selector, self.fix_toggle, self.angle_slider),
            self.plot_panel
        )

    def _precompute_data(self):
        heatmaps = {m: np.zeros((len(self.θ_vals), len(self.θ_vals))) for m in self.metrics}
        for i, θ1 in enumerate(self.θ_vals):
            for j, θ2 in enumerate(self.θ_vals):
                U, V = generate_subspaces(θ1, θ2)
                d = compute_distance_metrics(U, V)
                for m in self.metrics:
                    heatmaps[m][j, i] = d[m]  # row=θ₂, col=θ₁
        return heatmaps

    def _make_plot(self, selected_metric, fix_label, fixed_angle):
        θ_vals = self.θ_vals
        idx = int(np.clip(np.round(fixed_angle), 0, 90))

        # HEATMAP: show selected metric
        Z = self.data[selected_metric].T
        heatmap = hv.Image((θ_vals, θ_vals, Z), kdims=['θ₁ (°)', 'θ₂ (°)'], vdims=[selected_metric]).opts(
            cmap='viridis', colorbar=True, width=400, height=400,
            title=f"{selected_metric.capitalize()} distance over (θ₁, θ₂)"
        )

        # SLICE line
        if fix_label == 'θ₁ fixed':
            line = hv.VLine(fixed_angle).opts(color='red', line_width=2)
        else:
            line = hv.HLine(fixed_angle).opts(color='red', line_width=2)

        heatmap_with_line = heatmap * line

        # CURVE: all 4 metrics across sweep
        curves = []
        for m in self.metrics:
            Zm = self.data[m]
            ys = Zm[:, idx] if fix_label == 'θ₁ fixed' else Zm[idx, :]
            x_axis = θ_vals
            xlabel = 'θ₂ (°)' if fix_label == 'θ₁ fixed' else 'θ₁ (°)'
            curve = hv.Curve((x_axis, ys), xlabel, "Distance", label=m).opts(ylim=(0,2.3))
            curves.append(curve)

        curve_overlay = hv.Overlay(curves).opts(
            #title=f"All Metrics vs. {'θ₂' if fix_label == 'θ₁ fixed' else 'θ₁'}",
            title=f"Metric Slices at Fixed {'θ₁' if fix_label == 'θ₁ fixed' else 'θ₂'} = {fixed_angle}°",
            legend_position='right',
            width=500,
            height=400
        )

        return pn.Row(heatmap_with_line, curve_overlay)

    def __panel__(self):
        return self.layout

#hv.extension('bokeh', logo=False)
#CombinedDistanceViewer().servable()

In [27]:
def generate_rotated_data(angle_deg=0, n_samples=100, noise=0.05):
    angle_rad = np.radians(angle_deg)
    rotation = np.array([
        [np.cos(angle_rad), -np.sin(angle_rad)],
        [np.sin(angle_rad),  np.cos(angle_rad)]
    ])
    base = np.random.randn(n_samples, 2) @ np.diag([1.0, 0.2])
    rotated = base @ rotation.T
    noisy = rotated + noise * np.random.randn(*rotated.shape)
    return noisy

# Utility: signed angle between two vectors
def signed_angle(u, v):
    angle = np.arccos(np.clip(np.dot(u, v), -1.0, 1.0))
    cross = u[0]*v[1] - u[1]*v[0]
    return np.degrees(angle) * np.sign(cross)

class PCADriftViewer(pn.viewable.Viewer):
    def __init__(self, **params):
        super().__init__(**params)

        self.angle_slider = pn.widgets.FloatSlider(name="Rotation (degrees)", width=200,
                                                   start=-90, end=90, step=1, value=0)
        self.metric_selector = pn.widgets.Select(name="Metric", options=["Angle", "Spectral", "Chordal", "Geodesic"],
                                                 width=120)

        self.plot_pane = pn.pane.HoloViews(height=350, width=350)
        self.info_pane = pn.pane.Markdown(width=280)
        self.metric_curve_pane = pn.pane.HoloViews(height=250, width=280)

        self.reset_button = pn.widgets.Button(name="Reset", width=100)
        self.reset_button.on_click(self._on_reset)

        self._init_data()
        self.angle_slider.param.watch(self._update, 'value')
        self.metric_selector.param.watch(self._update, 'value')

        self._update()

    def _init_data(self):
        self.start_time = time.time()
        self.elapsed_times = []
        self.angle_history = []
        self.spectral_history = []
        self.chordal_history = []
        self.geodesic_history = []

        self.initial_data = generate_rotated_data(0)
        self.U0 = PCA(n_components=1).fit(self.initial_data).components_.T

    def _on_reset(self, *events):
        self._init_data()
        self._update()

    def _update(self, *events):
        theta = self.angle_slider.value
        data = generate_rotated_data(theta)
        U_t = PCA(n_components=1).fit(data).components_.T

        # Time tracking
        now = time.time()
        elapsed = now - self.start_time
        self.elapsed_times.append(elapsed)

        # Distance metrics
        angle_deg = signed_angle(self.U0[:, 0], U_t[:, 0])
        angles_rad = subspace_angles(self.U0, U_t)

        spectral = np.degrees(angles_rad[0])
        chordal  = np.linalg.norm(np.sin(angles_rad))
        geodesic = np.linalg.norm(np.degrees(angles_rad))

        self.angle_history.append(angle_deg)
        self.spectral_history.append(spectral)
        self.chordal_history.append(chordal)
        self.geodesic_history.append(geodesic)

        # Visual arc
        arc_theta = np.linspace(0, np.radians(angle_deg), 50)
        arc_x = np.cos(arc_theta)
        arc_y = np.sin(arc_theta)
        arc = hv.Curve((arc_x, arc_y)).opts(color='red', line_width=2)

        arrow_U0 = hv.Segments([(0, 0, *self.U0[:, 0])]).opts(line_color='gray', line_width=3)
        arrow_Ut = hv.Segments([(0, 0, *U_t[:, 0])]).opts(line_color='red', line_width=3)
        scatter  = hv.Points(data).opts(size=4, color='blue', alpha=0.5)

        plot = (scatter * arrow_U0 * arrow_Ut * arc).opts(
            aspect='equal', xlim=(-3, 3), ylim=(-3, 3), title="PCA Drift"
        )
        self.plot_pane.object = plot

        self.info_pane.object = f"""
<span style="font-size:14px">
<b>Principal Angle</b>: {angle_deg:+.2f}°<br>
<b>Rotation Input</b>: {theta:+.0f}°<br><br>
- Red: current PCA direction<br>
- Red arc: signed principal angle
</span>
"""

        if len(self.elapsed_times) >= 2:
            selected = self.metric_selector.value
            if selected == "Angle":
                yvals = self.angle_history
                ylabel = "Angle [deg]"
            elif selected == "Spectral":
                yvals = self.spectral_history
                ylabel = "Spectral Distance [deg]"
            elif selected == "Chordal":
                yvals = self.chordal_history
                ylabel = "Chordal Distance"
            else:
                yvals = self.geodesic_history
                ylabel = "Geodesic Distance [deg]"

            curve = hv.Curve((self.elapsed_times, yvals), 'Time [s]', ylabel).opts(
                color='crimson', height=250, tools=['hover'], show_grid=True,
                title=f"{selected} vs Time"
            )
            self.metric_curve_pane.object = curve

    def __panel__(self):
        return pn.Column(
            "## PCA Drift Demo (Metric Tracking)",
            pn.Row(self.reset_button, self.angle_slider, pn.Spacer(width=50), self.metric_selector),
            pn.Row(
                self.plot_pane,
                pn.Spacer(width=20),
                pn.Column(
                    pn.Spacer(height=10),
                    self.info_pane,
                    self.metric_curve_pane
                )
            )
        )


# 
<div style="float:center;width:100%;text-align:center;">
<strong style="height:100px;color:darkred;font-size:40px;">Computing Principal Angles</strong><br>
<strong style="height:100px;color:darkred;font-size:25px;">Algorithms and Selected Examples</strong></div>

# 1. Review: Principal Angles Between Subspaces

**Previous notebooks in this series:**
* [**HyperplanesPrincipalAngles.ipynb**](HyperplanesPrincipalAngles.ipynb)
* [**HyperplaneDistances.ipynb**](HyperplaneDistances.ipynb)

**Principal Angles Between Subspaces**

* Let $\mathscr{C}(U), \mathscr{C}(V) \subset \mathbb{R}^n$ be subspaces of dimension $k$ and $\ell$, respectively.<br>
The **principal angles** $\theta_1 \leq \cdots \leq \theta_r$, where $r = \min(k, \ell)$,<br>
measure the smallest angles between directions in the two subspaces.

* If the columns of both $U$ and $V$ are mutually orthogonal and unit length,<br>
i.e., if they are **orthonormal basis matrices**,<br>
then the principal angles are determined by the **singular values of** $\mathbf{U^T V}$<br><br>
If $\{ \sigma_i, i=1,\dots,r \}\;$ are the singular values of $U^T V$, the principal angles are given by<br>
$\qquad \theta_i = \cos^{-1}(\sigma_i)$.

This formulation is the most practical and numerically stable for computation.

---
**Focus of This Notebook**

This notebook examines the **algorithmic and numerical aspects** of principal angle computation. We go beyond the core definition to explore:

- The importance of **orthonormalizing basis matrices**
- Comparing **QR and SVD orthonormalization** strategies
- Behavior under **near-degenerate or ill-conditioned subspaces**
- The effect of numerical precision on **subspace distance metrics**

Small-scale demos, reproducible examples, and visual diagnostics are included to highlight robustness (or failure) in practice.

# 2. Algorithmic Choices in Computing Principal Angles

To compute principal angles between two subspaces with bases $A, B \in \mathbb{R}^{n \times k}$,<br>
we must **orthonormalize** the columns.

A poor orthonormalization step can distort angles — especially the small ones<br>
as we have already seen in the previous notebook.

## 2.1 Orthonormalization

### 2.1.1 Option 1: QR Decomposition

Fast and effective for well-conditioned matrices: compute the QR decomposition of the matrices $A$ and $B$.<br>
This is fast and numerically stable.

* $\qquad$ Decompose $\;A = Q R\;$ and set $U = Q$.
* $\qquad$ Decompose $\;B = Q R\;$ and set $V = Q$.

<div style="display: flex; margin-top: 1em; margin-bottom: 1em;">
<!-- ----------------------------------------- Python box -->
<div style="width: 22em; height: 4.8em; padding-right: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Python</div>
    <div>U, _ = np.linalg.qr(A)</div>
    <div>V, _ = np.linalg.qr(B)</div>
  </div>
</div>
<!-- ---------------------------------------- Julia box -->
<div style="width: 22em; height: 4.8em; margin-left: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Julia</div>
    <div>U = qr(A).Q</div>
    <div>V = qr(B).Q</div>
  </div>
</div>
</div>

### 2.1.2 Option 2: SVD-Based Orthonormalization

In this approach, we compute the **singular value decomposition** of the input matrices $A$ and $B$, and extract the left singular vectors to obtain orthonormal bases for their column spaces.

This method is **more robust** than QR when $A$ or $B$ are nearly rank-deficient or ill-conditioned. It is especially useful when numerical precision matters in detecting small principal angles.

* $\qquad$ Decompose $\;A = U \Sigma V^\top\;$ and set $U_{\text{orth}} = U$.
* $\qquad$ Decompose $\;B = U \Sigma V^\top\;$ and set $V_{\text{orth}} = U$.

<div style="display: flex; margin-top: 1em; margin-bottom: 1em;">

<!-- Python box -->
<div style="width: 10cm; height: 4.8em; padding-right: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Python</div>
    <div>U, _, _ = np.linalg.svd(A, full_matrices=False)</div>
    <div>V, _, _ = np.linalg.svd(B, full_matrices=False)</div>
  </div>
</div>

<!-- Julia box -->
<div style="width: 8cm; height: 4.8em; margin-left: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Julia</div>
    <div>U, _, _ = svd(A, full=false)</div>
    <div>V, _, _ = svd(B, full=false)</div>
  </div>
</div>

</div>

### 2.1.3 Numerical Rank and Orthonormalization

In principal angle computations, we typically start with $k$ vectors assumed to span a $k$-dimensional subspace.  
But if the vectors are nearly dependent—or if the rank is unknown—<br>the matrix may be **numerically rank-deficient**.

* **QR** always returns $k$ orthonormal vectors, assuming full rank.<br>
If the input is nearly rank-deficient:
    - The first QR vector may align with the subspace.
    - Remaining vectors are orthogonal by construction but **not geometrically meaningful**.
* **SVD** by contrast
    - Reveals numerical rank via singular values.
    - Retains only stable directions aligned with the data.

____
**QR** is fast and reliable when the input matrix has full rank and well-separated directions.<br>
$\qquad$ It returns an orthonormal basis where the first vector matches the first input column.<br>
$\qquad$ **Use QR for speed and simplicity when the geometry is well-conditioned.**

**SVD** gives a geometrically optimal basis, selecting directions<br>
$\qquad$ that span the column space as faithfully as possible.<br>
$\qquad$ It is especially useful when vectors are nearly dependent or rank-deficient.<br>
$\qquad$ **Use SVD when precision in subspace alignment matters, especially in degenerate or near-degenerate cases.**

____
We illustrate this with two examples:

- A computation where two nearly identical vectors yield spurious QR directions.<br>
  Discuss a thresholded QR strategy that retains only numerically stable directions.
- A parametric study of how QR and SVD behave as the angle between vectors increases.

#### Nearly Identical Vectors Example

In [12]:
ex_1()

Given vectors a1 and a2 are nearly identical:


QR returns two orthogonal vectors
    ‖QᵀQ − I‖∞      = 5.61e-16
    diagonal elements of R: -2.590802, 2.99e-14

SVD singular values reveal the near rank deficiency
    σ₁ = 3.66e+00,   σ₂ = 2.11e-14

Compare the angles of the QR vectors q₁ and q₂ with the singular vector u₁
    angle(q₁, u₁) = 0.000001°
    angle(q₂, u₁) = 90.000000°


____

#### Detecting Rank Deficiency

The diagonal entries of $R$ reflect the length of the vector components added at each step:<br>
* $R_{1 1} = \Vert a_1 \Vert$
* $R_{2 2} = \Vert a_2 - a_2\cdot q_1 q_1\Vert$ the length of the component of $a_2$ orthogonal to $q_1$.<br>
Small values indicate that the new direction contributes little new information — and should be discarded:<br>in the example above $R_{2 2} \approx 3\ 10^{-14}$

In practice, we might remove any orthogonal vectors $q_i$  from $A = Q R$ that have $R_{i i}$ below a chosen tolerance as shown below.

In [13]:
# remove any vectors that have an R_ii entry below tolerance
def truncated_qr(A, tol):
    Q, R   = np.linalg.qr(A)
    r_diag = np.abs(np.diag(R))
    return Q[:, r_diag > tol]

#### Theoretical Behavior for 2D Example

$\qquad$ Let $a_1, a_2 \in \mathbb{R}^n$, unit vectors with angle $\theta$ and set $A = (a_1 \;\; a_2) = \begin{pmatrix}1 & \cos\theta \\ 0 & \sin\theta \end{pmatrix}$.

$\qquad$ **QRdecomposition**<br>
$\qquad$ $\qquad A = \left(\begin{array}{rr}1 & 0 \\ 0 & 1 \end{array}\right)
\begin{pmatrix}1 & \cos\theta \\ 0 & \sin\theta \end{pmatrix},\qquad$ thus $R_{2 2} = \sin\theta$ 

$\qquad$ **Principal Values**<br>
$\qquad$ $\qquad A^T A = \begin{pmatrix} 1 & \cos\theta \\ \cos\theta & 1 \end{pmatrix},\quad$
has eigenvalues $\lambda = 1 \pm \cos\theta$  

$\qquad$ $\qquad$ Thus $\frac{\sigma_2}{\sigma_1} = \sqrt{\frac{1 - \cos\theta}{1 + \cos\theta}} = \tan(\theta/2).\qquad$  (Note this is the inverse of the condition number)

In [14]:
hv.extension('bokeh', logo=False)
pn.Column(
    "# QR vs SVD Diagnostics as Vector Angle Increase",
    qr_vs_svd_stacked_annotated()
).servable()

Column
    [0] Markdown(str)
    [1] Column
        [0] HoloViews(Overlay, height=100, sizing_mode='fixed', width=500)
        [1] HoloViews(Overlay, height=100, sizing_mode='fixed', width=500)
        [2] HoloViews(Overlay, height=120, sizing_mode='fixed', width=500)

**Conclusion**

As the angle $\theta$ between $a_1$ and $a_2$ increases, the subspace becomes more clearly 2-dimensional.  

- **QR** always returns $q_1 = a_1 / \|a_1\|$, preserving the input order.
- **SVD** selects $u_1$, the direction that best spans the full column space of $A$.

At small $\theta$, $a_1$ and $a_2$ are nearly aligned, so $q_1 \approx u_1$, and $\sigma_2 \ll \sigma_1$, $R_{22} \approx 0$.

As $\theta$ grows, the second direction becomes more significant:
- $u_1$ rotates to balance both vectors, reflecting the growing importance of the second direction
- $\angle(q_1, u_1)$ increases,
- $\sigma_2 / \sigma_1$ and $R_{22}$ both increase.

**Takeaway:**<br>
$\qquad$ R and SVD agree in the well-separated regime — but diverge geometrically when vectors are nearly dependent.

$\qquad$ For subspaces defined by nearly aligned vectors, **SVD more accurately captures the dominant direction**,<br>$\qquad$  while QR reflects the first input vector.<br>  
$\qquad$ As the separation grows, both methods yield similar bases.

## 2.2 Subspace Distances via Principal Angles

Now that we have orthonormal bases, i.e., the cols of $U \in \mathbb{R}^{M\times p},  \in \mathbb{R}^{M\times p}$,<br>
the principal angles are computed by a single SVD of $U^T V$.

$\qquad
\cos \theta_i = \sigma_i(U^T V)
$

These angles immediately determine all standard subspace distances (e.g., spectral, chordal, geodesic),<br>
as detailed in the [**previous notebook**](HyperplaneDistances.ipynb).


This is efficient, numerically stable, and widely used.

# 3. Applications

## 3.1 PCA Drift and Subspace Tracking

When applying PCA to streaming or time-evolving data, it is often useful to track how the dominant subspace changes over time.

Given a sequence of datasets $X_t \in \mathbb{R}^{m \times n}$, we compute PCA at each timestep to extract a $k$-dimensional subspace $U_t$.

To monitor drift, we compute the **principal angles** between $U_t$ and $U_{t+1}$. These angles — or distances derived from them — provide a quantitative measure of how much the underlying structure is changing.

For example:
- A small spectral distance $\theta_1$ indicates the top direction is stable.
- A large chordal or geodesic distance may indicate broader change.

This forms the basis for **concept drift detection**, **subspace tracking**, and **change point analysis**<br> in applications like signal processing, finance, and machine learning.

Below we construct a toy example showing how the distance between PCA subspaces changes as the data distribution rotates slowly.

In [28]:
hv.extension('bokeh', logo=False)
demo = PCADriftViewer()
demo.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] Button(name='Reset', width=100)
        [1] FloatSlider(end=90, name='Rotation (degrees)', start=-90, step=1, width=200)
        [2] Spacer(width=50)
        [3] Select(name='Metric', options=['Angle', 'Spectral', ...], value='Angle', width=120)
    [2] Row
        [0] HoloViews(Overlay, height=350, sizing_mode='fixed', width=350)
        [1] Spacer(width=20)
        [2] Column
            [0] Spacer(height=10)
            [1] Markdown(str, width=280)
            [2] HoloViews(None, height=250, sizing_mode='fixed', width=280)

**Example Applications using Subspace Drift**

Monitoring changes in principal angles between PCA subspaces provides a quantitative measure of structural shifts in evolving data. This approach is instrumental in various domains:

- **Streaming Data & Online Learning**: Detecting concept drift in real-time systems, such as recommendation engines and spam filters, where user behavior evolves over time.
- **Video Surveillance & Background Modeling**: Identifying scene changes or motion by tracking subspace variations in video frames.
- **Neuroscience & Biomedical Signals**: Monitoring shifts in EEG or fMRI data to detect phase transitions or anomalies in brain activity.
- **Financial Time Series Analysis**: Observing changes in market dynamics by analyzing shifts in principal components over different time periods.